In [1]:
import sys
import os
import cv2
sys.path.insert(0, '../src')

import torch
from torch import optim
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
from lovasz import lovasz_hinge

import matplotlib.pyplot as plt

In [2]:
from data_builder.builder import build_train_loader, build_valid_loader
from config import _C as cfg

In [3]:
data = os.listdir(os.path.join(cfg.DATA_DIR, 'train'))
train_lsit = list(set([row.split("_")[0] for row in data]))
train_idx = [row for row in data if row.split("_")[0] in train_lsit[:-2]]
valid_idx = [row for row in data if row.split("_")[0] not in train_lsit[:-2]]
len(train_idx),len(valid_idx)

(23, 2)

In [4]:
train_loader = build_train_loader(cfg, train_idx)
valid_loader = build_valid_loader(cfg, valid_idx)

TypeError: build_train_loader() takes 1 positional argument but 2 were given

In [5]:
valid_loader = build_valid_loader(cfg, valid_idx)

In [6]:
img, mask = next(iter(train_loader))

In [7]:
def symmetric_lovasz(outputs, targets):
    return 0.5*(lovasz_hinge(outputs, targets) + lovasz_hinge(-outputs, 1.0 - targets))

In [8]:
metric = smp.utils.losses.DiceLoss()

In [9]:
class hmapModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.model = smp.Unet('resnet34', encoder_weights='imagenet')
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = symmetric_lovasz(y_hat, y)
        dice_m = metric(y_hat, y)
        self.log('train_loss', loss)
        self.log('train_dice_loss', dice_m)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = metric(y_hat, y)
        self.log('valid_dice_loss', loss)

In [10]:
model = hmapModel()

In [ ]:
trainer = pl.Trainer(gpus=1)
trainer.fit(model, train_loader, valid_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
